In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import pickle
import joblib

import esipy as esi

from tqdm.notebook import tqdm, trange

import requests

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

Num GPUs Available:  1


In [3]:
# Set up network access
esi_client = esi.EsiClient(
    transport_adapter = requests.adapters.HTTPAdapter(
        pool_connections=100,
        pool_maxsize=100,
        max_retries=10,
        pool_block=False
    )
)
esi_app    = esi.EsiApp()

app = esi_app.get_latest_swagger

W0222 10:08:28.622508  6520 client.py:81] Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
d:\programme\python 3.6.6\lib\site-packages\esipy\client.py:82: UserWarning: Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
  warnings.warn(warning_message)


In [4]:
# Load systems
esi_op = app.op['get_universe_systems']()
response = esi_client.request(esi_op);
system_ids = response.data;

In [6]:
# Load data for all systems
ops = [app.op['get_universe_systems_system_id'](system_id = system_id) for system_id in system_ids];
responses = esi_client.multi_request(tqdm(ops));
systems = {k : resp[1].data for k,resp in zip(system_ids, responses)};

In [8]:
# Loading stargates
stargate_ids = [];

for system in systems.values():
    if 'stargates' not in system:
        continue;
        
    stargate_ids.extend(system.stargates);

ops = [app.op['get_universe_stargates_stargate_id'](stargate_id = stargate_id) for stargate_id in stargate_ids];
responses = esi_client.multi_request(tqdm(ops));
stargates = {k : resp[1].data for k,resp in zip(stargate_ids, responses)};

In [11]:
import json

# Save graph
with open('universe/stargates.txt', 'w') as f:
    json.dump(stargates, f, indent = 4)

In [17]:
def intd(f):
    return {int(k) : v for k,v in json.load(f).items()};

with open('universe/systems.txt') as f:
    systems = intd(f);

with open('universe/stargates.txt') as f:
    stargates = intd(f);

In [18]:
systems_graph = nx.Graph();

for system in systems:
    systems_graph.add_node(system);

for stargate in stargates.values():
    systems_graph.add_edge(stargate["system_id"], stargate["destination"]["system_id"])

In [10]:
import json

# Save graph
with open('universe/systems.txt', 'w') as f:
    json.dump(systems, f, indent = 4)

In [19]:
root = 'Jita'
root_id = [v for v in systems if systems[v]["name"] == root][0]

# Limit to high-sec systems
subselect = [k for k in systems if systems[k]["security_status"] > 0.65]
subgraph = systems_graph.subgraph(subselect)

# Limit to component connected to Jita
subselect = nx.single_source_shortest_path_length(systems_graph, root_id)
subgraph = systems_graph.subgraph(subselect)

In [11]:
# Landmark systems
landmark_names = ['Jita', 'Amarr', 'Rens', 'Hek', 'Dodixie', 'Oursulaert', 'Tash-Murkon Prime', 'Agil'];

landmark_ids = [v for v in systems if systems[v].name in landmark_names]

distances = [];

print('Processing landmarks');

# Compute distances from landmark systems
for landmark_id in tqdm(landmark_ids):
    # Compute distances in graph
    lengths = nx.single_source_shortest_path_length(systems_graph, landmark_id);
    
    for k,v in lengths.items():
        distances.append((landmark_id, k, v, 1.0));

Processing landmarks


In [12]:
# Compute all distances in a len(subgraph) x len(subgraph) array

distances = np.asarray([
    [d for j,d in sorted(nx.single_source_shortest_path_length(subgraph, i).items())]
    for i in tqdm(subgraph)
])

In [13]:
distances.shape

(5201, 5201)

In [90]:
# Set up the model

embedding_dimension = 7;

# Embedding for the systems
positions = tf.Variable(
    np.random.normal(size = [len(subgraph), embedding_dimension]),
    dtype = tf.float32
)

# Tensor that selects the pairs of systems we have the distances for
# selector = tf.Variable(
#     [[system_ids.index(a), system_ids.index(b)] for a,b,d,w in distances],
#    dtype = tf.int32,
#    name = 'System ids'
#)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0


In [91]:
expected_distances = tf.constant(
    #[d for a,b,d,w in distances],
    distances,
    dtype = tf.float32,
    name = 'Input distances'
)

def calc_distance():
    # Gather into len(distances) x len([a,b])=2 x embedding_dimension array and calculate distances
    # gathered = tf.gather(positions, selector, axis = 0)
    #return tf.norm(gathered[:,1,:] - gathered[:,0,:], axis = -1)
    return tf.norm(
        tf.reshape(positions, [1, len(subgraph), embedding_dimension]) - tf.reshape(positions, [len(subgraph), 1, embedding_dimension]),
        axis = -1
    )

def loss_function():
    loss_fn = tf.keras.losses.MeanAbsoluteError()
    loss = loss_fn(expected_distances, calc_distance())
    
    return loss

In [94]:
def process(x):
    return x[::20, ::20].numpy().flatten()

plt.scatter(process(expected_distances), process(calc_distance()), marker = '.')
plt.axis('equal')
plt.plot([0, 100], [0, 100])
plt.xlabel('Real distance');
plt.ylabel('Calculated distance');
plt.xlim(0, 40)
plt.ylim(0, 40)

ResourceExhaustedError: OOM when allocating tensor with shape[5201,5201,7] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Mul] name: norm/mul/

In [92]:
optimizer = tf.keras.optimizers.Adam(1e-0)

@tf.function
def optimize():
    for i in tf.range(30):
        optimizer.minimize(loss_function, lambda: positions)

In [93]:
for i in trange(0, 100):
    optimize()
    print(loss_function().numpy())

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_optimize_23230 in device /job:localhost/replica:0/task:0/device:GPU:0


ResourceExhaustedError:  OOM when allocating tensor with shape[5201,5201,7] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node while/body/_1/norm/ArithmeticOptimizer/ReplaceMulWithSquare_mul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_optimize_23230]

Function call stack:
optimize


In [22]:
systems

{30000001: {'constellation_id': 20000001,
  'name': 'Tanoo',
  'planets': [{'asteroid_belts': [40000003],
    'moons': [40000004],
    'planet_id': 40000002},
   {'asteroid_belts': [40000006], 'planet_id': 40000005},
   {'planet_id': 40000007},
   {'asteroid_belts': [40000009], 'moons': [40000010], 'planet_id': 40000008},
   {'moons': [40000012, 40000013, 40000014, 40000015, 40000016],
    'planet_id': 40000011},
   {'asteroid_belts': [40000018], 'planet_id': 40000017}],
  'position': {'x': -8.851079259998058e+16,
   'y': 4.236944396687888e+16,
   'z': -4.451352534647966e+16},
  'security_class': 'B',
  'security_status': 0.8583240509033203,
  'star_id': 40000001,
  'stargates': [50000056, 50000057, 50000058],
  'stations': [60012526, 60014437],
  'system_id': 30000001},
 30000002: {'constellation_id': 20000001,
  'name': 'Lashesih',
  'planets': [{'moons': [40000021], 'planet_id': 40000020},
   {'asteroid_belts': [40000023], 'planet_id': 40000022},
   {'asteroid_belts': [40000025],
  